In [ ]:
# sales_ml_pipeline_no_employee.py

# ML pipeline for invoice-level sales data with columns:
# - INV No
# - Date
# - Customer Name
# - Document Total

# This version removes any Sales Employee logic (per user request) and focuses on:
# - Data cleaning & validation
# - Exploratory aggregation
# - Time-series forecasting (monthly) using Prophet if available, otherwise SARIMAX or moving average
# - Customer segmentation (RFM + KMeans)
# - Churn prediction (simple RFM-based)
# - Invoice value prediction using:
#     * Linear Regression (LR)
#     * Decision Tree Regressor (DT)
#     * Random Forest Regressor (RF) as a strong benchmark
# - Model comparison and saving results

# Usage:
#     python sales_ml_pipeline_no_employee.py --input "C:/path/to/INVOICE_DATA-MLR-DT-LR.xlsx" --output_dir ./output --date_col Date

# Notes:
# - Script auto-detects .xlsx/.xls and .csv
# - Required packages:
#     pandas numpy scikit-learn matplotlib joblib statsmodels prophet (optional)
# """

import os
import sys
import argparse
import warnings
from pathlib import Path
import json
import joblib

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV, TimeSeriesSplit, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings("ignore")
RANDOM_STATE = 42

# Optional forecasting libs
HAS_PROPHET = False
HAS_STATS = False
try:
    from prophet import Prophet
    HAS_PROPHET = True
except Exception:
    try:
        from fbprophet import Prophet
        HAS_PROPHET = True
    except Exception:
        HAS_PROPHET = False

try:
    import statsmodels.api as sm
    HAS_STATS = True
except Exception:
    HAS_STATS = False

def read_data(path, date_col='Date'):
    path = str(path)
    if path.lower().endswith(('.xls', '.xlsx')):
        df = pd.read_excel(path)
    else:
        df = pd.read_csv(path, low_memory=False)
    if date_col not in df.columns:
        raise ValueError(f"Date column '{date_col}' not found in input. Columns: {df.columns.tolist()}")
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    df = df.rename(columns=lambda x: x.strip())
    for c in df.select_dtypes(include='object').columns:
        df[c] = df[c].astype(str).str.strip()
    return df

def basic_clean(df):
    # Ensure Document Total numeric
    if 'Document Total' not in df.columns:
        possible = [c for c in df.columns if 'document' in c.lower() and 'total' in c.lower()]
        if possible:
            df = df.rename(columns={possible[0]:'Document Total'})
        else:
            raise ValueError("Document Total column not found.")
    df['Document Total'] = pd.to_numeric(df['Document Total'], errors='coerce').fillna(0.0)
    if 'INV No' not in df.columns:
        df['INV No'] = pd.Series(np.arange(1, len(df)+1)).astype(str)
    df['INV No'] = df['INV No'].astype(str)
    if 'Customer Name' not in df.columns:
        df['Customer Name'] = 'UNKNOWN'
    df = df.dropna(subset=['Date'])
    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month
    df['Day'] = df['Date'].dt.day
    df['YearMonth'] = df['Date'].dt.to_period('M').astype(str)
    return df

def aggregate_time_series(df, freq='M'):
    ts = df.set_index('Date').resample(freq)['Document Total'].sum().reset_index()
    ts = ts.rename(columns={'Date':'ds', 'Document Total':'y'})
    return ts

def forecast(ts, periods=12, method_preference='prophet'):
    if method_preference == 'prophet' and HAS_PROPHET:
        m = Prophet()
        m.fit(ts)
        future = m.make_future_dataframe(periods=periods, freq='M')
        forecast = m.predict(future)
        return 'prophet', m, forecast
    elif HAS_STATS:
        # SARIMAX fallback
        ts2 = ts.set_index('ds').asfreq('M')
        ts2['y'] = ts2['y'].fillna(0)
        try:
            model = sm.tsa.statespace.SARIMAX(ts2['y'], order=(1,1,1), seasonal_order=(1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
            res = model.fit(disp=False)
            pred = res.get_forecast(steps=periods)
            pred_index = pd.date_range(start=ts2.index[-1] + pd.offsets.MonthBegin(1), periods=periods, freq='M')
            forecast = pd.DataFrame({'ds': pred_index, 'yhat': pred.predicted_mean.values})
            return 'sarimax', res, forecast
        except Exception as e:
            print("SARIMAX failed:", e)
    # naive moving average
    ts['ma3'] = ts['y'].rolling(3, min_periods=1).mean()
    last = ts['ma3'].iloc[-1]
    future_idx = pd.date_range(start=ts['ds'].iloc[-1] + pd.offsets.MonthBegin(1), periods=periods, freq='M')
    forecast = pd.DataFrame({'ds': future_idx, 'yhat': np.repeat(last, periods)})
    return 'naive', None, forecast

def rfm_features(df, snapshot_date=None):
    if snapshot_date is None:
        snapshot_date = df['Date'].max() + pd.Timedelta(days=1)
    rfm = df.groupby('Customer Name').agg({
        'Date': lambda x: (snapshot_date - x.max()).days,
        'INV No': 'count',
        'Document Total': 'sum'
    }).rename(columns={'Date':'Recency', 'INV No':'Frequency', 'Document Total':'Monetary'})
    rfm = rfm.reset_index()
    return rfm

def customer_segmentation(rfm, n_clusters=4):
    rfm2 = rfm.copy()
    rfm2['Monetary'] = np.log1p(rfm2['Monetary'])
    X = rfm2[['Recency','Frequency','Monetary']].fillna(0)
    from sklearn.cluster import KMeans
    kmeans = KMeans(n_clusters=n_clusters, random_state=RANDOM_STATE)
    rfm2['Cluster'] = kmeans.fit_predict(X)
    return rfm2, kmeans

def create_churn_label(df, days_window=90):
    last_date = df['Date'].max()
    cutoff = last_date - pd.Timedelta(days=days_window)
    recent = df[df['Date'] > cutoff]['Customer Name'].unique()
    customers = df['Customer Name'].unique()
    labels = pd.DataFrame({'Customer Name': customers})
    labels['churn'] = (~labels['Customer Name'].isin(recent)).astype(int)
    return labels

def anomaly_detection(df):
    df2 = df.copy()
    cust_avg = df.groupby('Customer Name')['Document Total'].mean().rename('cust_mean')
    df2 = df2.join(cust_avg, on='Customer Name')
    df2['ratio'] = df2['Document Total'] / (df2['cust_mean'] + 1e-6)
    from sklearn.ensemble import IsolationForest
    iso = IsolationForest(contamination=0.01, random_state=RANDOM_STATE)
    feats = ['Document Total','ratio']
    df2['anomaly'] = iso.fit_predict(df2[feats].fillna(0))
    df2['is_anomaly'] = (df2['anomaly'] == -1).astype(int)
    return df2[['INV No','Date','Customer Name','Document Total','is_anomaly']]

def prepare_regression_features(df):
    d = df.copy()
    cust_avg = df.groupby('Customer Name')['Document Total'].agg(['mean','count']).rename(columns={'mean':'cust_avg','count':'cust_count'})
    d = d.join(cust_avg, on='Customer Name')
    d['month'] = d['Date'].dt.month
    d['dayofweek'] = d['Date'].dt.dayofweek
    d['is_month_start'] = d['Date'].dt.is_month_start.astype(int)
    d['is_month_end'] = d['Date'].dt.is_month_end.astype(int)
    # Encode Customer Name simple
    le = LabelEncoder()
    d['Customer_enc'] = le.fit_transform(d['Customer Name'].astype(str))
    features = ['cust_avg','cust_count','month','dayofweek','is_month_start','is_month_end','Customer_enc']
    X = d[features].fillna(0)
    y = d['Document Total']
    return X, y, d, le

def train_and_compare_models(X, y, outdir):
    results = {}
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)
    # Linear Regression
    lr = LinearRegression()
    lr.fit(X_train, y_train)
    pred_lr = lr.predict(X_test)
    results['LR'] = {'model': lr, 'rmse': mean_squared_error(y_test, pred_lr, squared=False), 'r2': r2_score(y_test, pred_lr)}
    # Decision Tree with simple grid search
    dt = DecisionTreeRegressor(random_state=RANDOM_STATE)
    param_grid = {'max_depth':[3,5,7,None], 'min_samples_split':[2,5,10]}
    gs = GridSearchCV(dt, param_grid, cv=3, scoring='neg_root_mean_squared_error', n_jobs=-1)
    gs.fit(X_train, y_train)
    best_dt = gs.best_estimator_
    pred_dt = best_dt.predict(X_test)
    results['DT'] = {'model': best_dt, 'rmse': mean_squared_error(y_test, pred_dt, squared=False), 'r2': r2_score(y_test, pred_dt)}
    # Random Forest benchmark
    rf = RandomForestRegressor(n_estimators=200, random_state=RANDOM_STATE, n_jobs=-1)
    rf.fit(X_train, y_train)
    pred_rf = rf.predict(X_test)
    results['RF'] = {'model': rf, 'rmse': mean_squared_error(y_test, pred_rf, squared=False), 'r2': r2_score(y_test, pred_rf)}
    # Save metrics to CSV
    metrics = []
    for name, info in results.items():
        metrics.append({'model': name, 'rmse': float(info['rmse']), 'r2': float(info['r2'])})
        joblib.dump(info['model'], os.path.join(outdir, f"model_{name}.joblib"))
    pd.DataFrame(metrics).to_csv(os.path.join(outdir, "model_comparison_metrics.csv"), index=False)
    return results, X_test, y_test

def save_fig(fig, path):
    fig.savefig(path, bbox_inches='tight', dpi=150)
    plt.close(fig)

def main(args):
    input_path = args.input
    outdir = Path(args.output_dir)
    outdir.mkdir(parents=True, exist_ok=True)
    print("Reading data...", input_path)
    df = read_data(input_path, date_col=args.date_col)
    df = basic_clean(df)
    print(f"Rows: {len(df)}; date range: {df['Date'].min().date()} to {df['Date'].max().date()}")
    # Time series
    ts_monthly = aggregate_time_series(df, freq='M')
    ts_monthly.to_csv(outdir / "monthly_timeseries.csv", index=False)
    fig, ax = plt.subplots(figsize=(10,4))
    ax.plot(ts_monthly['ds'], ts_monthly['y'], marker='o')
    ax.set_title('Monthly Sales (Document Total)')
    save_fig(fig, outdir / "monthly_sales.png")
    # Forecast
    method, model_obj, forecast_df = forecast(ts_monthly, periods=args.forecast_periods, method_preference='prophet')
    if forecast_df is not None:
        if 'yhat' in forecast_df.columns:
            forecast_df.to_csv(outdir / f"{method}_forecast.csv", index=False)
        else:
            forecast_df.to_csv(outdir / f"{method}_forecast.csv", index=False)
    # RFM & segmentation
    rfm = rfm_features(df)
    rfm.to_csv(outdir / "rfm.csv", index=False)
    rfm2, kmeans = customer_segmentation(rfm, n_clusters=args.n_clusters)
    rfm2.to_csv(outdir / "rfm_segmented.csv", index=False)
    # Churn labels
    labels = create_churn_label(df, days_window=args.churn_days_window)
    labels.to_csv(outdir / "churn_labels.csv", index=False)
    # Anomalies
    anomalies = anomaly_detection(df)
    anomalies.to_csv(outdir / "anomalies.csv", index=False)
    # Regression models (LR, DT, RF)
    X, y, df_feats, cust_le = prepare_regression_features(df)
    results, X_test, y_test = train_and_compare_models(X, y, outdir)
    # Save sample predictions
    best_model_name = min(results.keys(), key=lambda k: results[k]['rmse'])
    best_model = results[best_model_name]['model']
    df_feats['predicted_invoice_value_'+best_model_name] = best_model.predict(X)
    df_feats[['INV No','Date','Customer Name','Document Total','predicted_invoice_value_'+best_model_name]].to_csv(outdir / "invoice_predictions.csv", index=False)
    # Save encoders and kmeans
    joblib.dump(cust_le, outdir / "customer_labelencoder.joblib")
    joblib.dump(kmeans, outdir / "customer_kmeans.joblib")
    # Report
    report = {
        'rows': int(len(df)),
        'date_start': str(df['Date'].min().date()),
        'date_end': str(df['Date'].max().date()),
        'total_revenue': float(df['Document Total'].sum()),
        'best_model': best_model_name,
        'model_metrics': {k:{'rmse':float(v['rmse']), 'r2':float(v['r2'])} for k,v in results.items()}
    }
    with open(outdir / "report.json", 'w') as f:
        json.dump(report, f, indent=2)
    print("All outputs saved to", outdir)

if __name__ == '__main__':
    parser = argparse.ArgumentParser(description="Sales ML pipeline (no employee fields)")
    parser.add_argument('--input', required=True, help='Path to Excel (.xlsx/.xls) or CSV file')
    parser.add_argument('--output_dir', default='./output', help='Directory to save outputs')
    parser.add_argument('--date_col', default='Date', help='Name of the date column')
    parser.add_argument('--forecast_periods', type=int, default=12, help='Months to forecast')
    parser.add_argument('--n_clusters', type=int, default=4, help='Number of customer clusters')
    parser.add_argument('--churn_days_window', type=int, default=90, help='Days window for churn labeling')
    args = parser.parse_args()
    main(args)
# # write to file
# output_path = "/mnt/data/sales_ml_pipeline_no_employee.py"
# with open(output_path, "w", encoding="utf-8") as f:
#     f.write(script_content)

# print(f"Script written to {output_path}")
# print("Download: sandbox:" + output_path)
